In [2]:
import sys
sys.path.append("..")

In [9]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

load_dotenv()

llm = ChatGroq(
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model_name="openai/gpt-oss-120b",
    temperature=0.6
)

In [10]:
from langchain_community.utilities import SQLDatabase

from sql_alchemy.engine import engine
 
db = SQLDatabase(engine)

In [11]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

## Way 1 (old, shortcut)

In [6]:
from langchain_community.agent_toolkits.sql.base import create_sql_agent

sql_agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

question = "How many tshirts do we have left for nike in extra small size and white color?"

response = sql_agent. invoke( {"input": question})

print("\nFinal Answer:")
print(response["output"])



> Entering new SQL Agent Executor chain...
<think>
Okay, let's tackle this question. The user is asking how many t-shirts they have left for Nike in extra small size and white color. 

First, I need to figure out which tables in the database are relevant. Since they mentioned t-shirts, Nike, size, and color, the most likely table is something like a products or inventory table. But to be sure, I should list all the tables in the database.

I'll start by using the sql_db_list_tables tool. That will give me a list of all tables. Let's see what tables exist. Maybe there's a table named 'products', 'inventory', 'tshirts', or similar.

Once I have the list of tables, I should check the schema of the most relevant one. For example, if there's a 'products' table, I'll look at its columns to see if it includes brand, size, color, and quantity. The user needs to filter by brand (Nike), size (extra small), color (white), and sum the quantity.

If the schema shows that the 'products' table has 

## New api

In [ ]:
from langchain.agents import create_agent

system_message = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.

IMPORTANT DATABASE RULES:

- Column values such as brand, color, and size are ENUMs and are CASE-SENSITIVE.
- You MUST inspect the table schema before filtering on ENUM columns.
- You MUST match user input to the closest valid ENUM value from the database.
  Example: "white" → "White", "nike" → "Nike", "extra small" → "XS".
- If a query returns zero rows, you MUST re-check casing and ENUM values before answering.
- You MUST NEVER answer with brands, colors, or sizes that were not mentioned by the user.
- If the database does not contain matching rows after verification, say so explicitly.
- Do NOT guess. Do NOT generalize. Do NOT switch brands or sizes.
""".format(
    dialect=db.dialect,
    top_k=5,
)

agent = create_agent(llm, toolkit.get_tools(), system_prompt=system_message)

question = "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?"

events = agent.stream(
    {"messages": [("user", question)]},
    stream_mode="values",
)

for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (fc_bfc9b9fe-e826-42e5-8f70-1bd7caaad420)
 Call ID: fc_bfc9b9fe-e826-42e5-8f70-1bd7caaad420
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

discounts, t_shirts
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (fc_ee20054e-6b45-46eb-8ba2-fa6d3a25c085)
 Call ID: fc_ee20054e-6b45-46eb-8ba2-fa6d3a25c085
  Args:
    table_names: discounts, t_shirts
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE discounts (
	discount_id SERIAL NOT NULL, 
	t_shirt_id INTEGER NOT NULL, 
	pct_disc